In [4]:
import numpy as np
import sklearn as sk
import pmdarima
import matplotlib.pyplot as plt
import seaborn as sns
def interpolate_daily_prices_from_df(monthly_prices_df):
    daily_dates = pd.date_range(start=monthly_prices_df.index[0], end=monthly_prices_df.index[-1], freq='D')
    daily_prices_df = pd.DataFrame(index=daily_dates)

    for column in monthly_prices_df.columns:
        interpolated_prices = np.interp(
            x=daily_dates.astype(np.int64) // 10**9,  # Convert to Unix timestamp
            xp=monthly_prices_df.index.astype(np.int64) // 10**9,  # Convert to Unix timestamp
            fp=monthly_prices_df[column].values
        )
        daily_prices_df[column] = interpolated_prices

    return daily_prices_df

In [13]:
    
def get_price(date):
        
    from pandas import read_csv
    d = read_csv("Nat_Gas.csv")
    from pandas import to_datetime
    d['Dates'] = to_datetime(d['Dates'])
    
    train = d.iloc[:len(d) - 12 ]
    test = d.iloc[len(d)-12:] 
      
    from statsmodels.tsa.statespace.sarimax import SARIMAX 
      
    #result = SARIMAX(train['Prices'],  
                    #order = (0, 0, 0),  
                    #seasonal_order =(2, 1, 0, 12)).fit()
       
    
    forecast = SARIMAX(train['Prices'],  
                    order = (0, 0, 0),  
                    seasonal_order =(2, 1, 0, 12)).fit().predict(start = (len(d)),  
                               end = (len(d)-1) +  12,  
                               typ = 'levels')
    
    d.loc[:, ['Prices']] = d.loc[:, ['Prices']].interpolate(method='linear', limit_direction='both', limit=1000)
    
    from pandas import date_range 
    from pandas import concat
    
    
    df = []
    pred = date_range(start='10/31/2024', end='09/30/2025', freq='M')
    b = 48
    for i in pred:
        df.append([i,forecast[b]])
        b+=1
    df = pd.DataFrame(df, columns = ['Dates','Prices'])
    
    d = concat([d,df])
    d.set_index('Dates',inplace=True)
    d = interpolate_daily_prices_from_df(d)
    date = to_datetime(date)
    return d._get_value(date, 'Prices') 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
result = seasonal_decompose(d['Prices'],  
                            model ='additive', period = 12) 
  
# ETS plot  
result.plot()

from pmdarima import auto_arima 
  
# # Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 
  
# # Fit auto_arima function to AirPassengers dataset 
stepwise_fit = auto_arima(d['Prices'], start_p = 0, start_q = 0, 
                           max_p = 365, max_q = 365, m = 12, 
                           start_P = 0, seasonal = True, 
                           d = None, D = 1, trace = True, 
                           error_action ='ignore',   # we don't want to know if an order does not work 
                           suppress_warnings = True,  # we don't want convergence warnings 
                          stepwise = True)  
stepwise_fit 

In [ ]:

start = len(train)
end = len(d) - 1

# Predictions for one-year against the test set 
predictions = result.predict(start, end, 
                             typ = 'levels').rename("Predictions") 
  
# plot predictions and actual values 
predictions.plot(legend=True) 
test['Prices'].plot(legend=True)
# d['Prices'].plot(legend=True)

In [ ]:
from sklearn.metrics import mean_squared_error 
from statsmodels.tools.eval_measures import rmse 


# Calculate root mean squared error 
print(rmse(test["Prices"], predictions)," , ",
mean_squared_error(test["Prices"], predictions))  


In [15]:
date = input("Enter date: ")
get_price(date)

Enter date:  2025/02/06


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.27153D+00    |proj g|=  8.31801D+00

At iterate    5    f=  5.18309D-01    |proj g|=  3.14874D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      9     17      1     0     0   1.449D-06   5.165D-01
  F =  0.51653774050569934     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


/var/folders/ms/pqgh2n056hb02_t21cf9s0qw0000gn/T/ipykernel_60342/4230953852.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['Dates'] = to_datetime(d['Dates'])
/Users/rikkumar/miniconda3/envs/py3117/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.
/Users/rikkumar/miniconda3/envs/py3117/lib/python3.11/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/var/folders/ms/pqgh2n056hb02_t21cf9s0qw0000g

12.482927759726573